In [ ]:
!pip install pandas_datareader

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from numpy.core.fromnumeric import shape
from numpy.lib.type_check import real
import pandas as pd
import pandas_datareader as pdr
import datetime as dt
from sklearn import preprocessing
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.layers import Dense,Dropout,LSTM
from tensorflow.keras.models import Sequential
from sklearn.model_selection import train_test_split
from tensorflow import keras

crypto='BTC'
currency='USD'

In [2]:
start=dt.datetime(2020,1,1)
end=dt.datetime(2022,7,1)

data=pdr.DataReader(f'{crypto}-{currency}','yahoo',start,end)

In [3]:
data.describe()

,High,Low,Open,Close,Volume,Adj Close
count,914.000000,914.000000,914.000000,914.000000,9.140000e+02,914.000000
mean,31521.398876,29833.744328,30731.332863,30739.522253,3.789194e+10,30739.522253
std,18649.168285,17632.569706,18183.830957,18165.409555,1.950139e+10,18165.409555
min,5331.833984,4106.980957,5002.578125,4970.788086,1.225260e+10,4970.788086
25%,10798.186279,10530.265869,10676.835693,10683.807617,2.610501e+10,10683.807617
50%,34778.076172,32265.797852,33767.375000,33734.515625,3.412977e+10,33734.515625
75%,47144.067383,44450.456055,46153.047852,46164.187500,4.479641e+10,46164.187500
max,68789.625000,66382.062500,67549.734375,67566.828125,3.509679e+11,67566.828125


In [4]:
scaler=MinMaxScaler(feature_range=(0,1))
scaled_data=scaler.fit_transform(data['Close'].values.reshape(-1,1))

prediction_days=30
future_day=0

In [5]:
data

,High,Low,Open,Close,Volume,Adj Close
Date,,,,,,
2020-01-01,7254.330566,7174.944336,7194.892090,7200.174316,18565664997,7200.174316
2020-01-02,7212.155273,6935.270020,7202.551270,6985.470215,20802083465,6985.470215
2020-01-03,7413.715332,6914.996094,6984.428711,7344.884277,28111481032,7344.884277
2020-01-04,7427.385742,7309.514160,7345.375488,7410.656738,18444271275,7410.656738
2020-01-05,7544.497070,7400.535645,7410.451660,7411.317383,19725074095,7411.317383
...,...,...,...,...,...,...
2022-06-28,21164.423828,20228.812500,20731.544922,20280.634766,21381535161,20280.634766
2022-06-29,20364.156250,19937.791016,20281.169922,20104.023438,23552740328,20104.023438
2022-06-30,20141.160156,18729.656250,20108.312500,19784.726562,26267239923,19784.726562


In [6]:
X=np.array(data.drop('Close',axis=1))

Y=np.array(data['Close'])

In [7]:
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.33, random_state=42)


In [8]:
x_train=np.reshape(x_train,(x_train.shape[0],x_train.shape[1],1))


In [9]:
x_train.shape

(612, 5, 1)

In [10]:
y_train.shape

(612,)

In [11]:
model=Sequential()
model.add(LSTM(units=50,return_sequences=True,input_shape=(x_train.shape[1],1)))
model.add(Dropout(0.2))
model.add(LSTM(units=50,return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(units=50))
model.add(Dense(units=1))
model.compile(loss = 'mean_squared_error',
optimizer = 'adam',
metrics = ['accuracy'])
callback = keras.callbacks.EarlyStopping(monitor='loss',
                                            patience=10,
                                            restore_best_weights=True)

In [12]:
model.fit(x_train,y_train,epochs=30,batch_size=4, callbacks=callback)

Epoch 1/30
153/153 [==============================] - 4s 6ms/step - loss: 1304139264.0000 - accuracy: 0.0000e+00
Epoch 2/30
153/153 [==============================] - 1s 6ms/step - loss: 1303527424.0000 - accuracy: 0.0000e+00
Epoch 3/30
153/153 [==============================] - 1s 6ms/step - loss: 1303037952.0000 - accuracy: 0.0000e+00
Epoch 4/30
153/153 [==============================] - 1s 6ms/step - loss: 1302558080.0000 - accuracy: 0.0000e+00
Epoch 5/30
153/153 [==============================] - 1s 5ms/step - loss: 1302083328.0000 - accuracy: 0.0000e+00
Epoch 6/30
153/153 [==============================] - 1s 5ms/step - loss: 1301612544.0000 - accuracy: 0.0000e+00
Epoch 7/30
128/153 [========================>.....] - ETA: 0s - loss: 1305113856.0000 - accuracy: 0.0000e+00

KeyboardInterrupt: 

In [ ]:
loss, accuracy = model.evaluate((x_val,y_val))
print("Loss: ", loss)
print("Accuracy: ", accuracy)

In [ ]:
test_start=dt.datetime(2020,1,1)
test_end=dt.datetime.now()

test_data=pdr.DataReader(f'{crypto}-{currency}','yahoo',test_start,test_end)

actual_prices=test_data['Close'].values

total_dataset=pd.concat((data['Close'],test_data['Close']),axis=0)

model_inputs=total_dataset[len(total_dataset)-len(test_data)-prediction_days:].values
model_inputs=model_inputs.reshape(-1,1)

model_inputs=scaler.fit_transform(model_inputs)

x_test=[]

for x in range(prediction_days,len(model_inputs)):
    x_test.append(model_inputs[x-prediction_days:x,0])

x_test=np.array(x_test)
x_test=np.reshape(x_test,(x_test.shape[0],x_test.shape[1],1))